# Training the Agent
In this notebook, instructions are provided for running the parallelized agent training process. Users should read along in this notebook, but enter the commands found here into a terminal window (non-administrator for Windows).

## Parallel Training using Ray
Now that we have seen that our environment will work with ARS, it is time to do some serious training. Running only one car at a time at 15 seconds per episode would take forever to train, so we want to be able to have multiple cars exploring deltas simultaneously and pooling their results together for the update step. To do this, I have altered [the code made available by Mania, Guy, and Recht to reproduce their research on MuJoCo locomotion tasks](https://github.com/modestyachts/ARS) in order to implement the CarEnv instead. Their ARS framework includes functionality on a Ray server. 

In order for this to work, the reader must have ray installed using pip install ray. For Windows, use a non-administrator Powershell terminal using the same environment as this notebook to call 'ray start --head' as well as any parameters that are appropriate for your machine. The CarEnvs take up a lot of RAM, so it may be necessary to set your --object-store-memory and --memory parameters to as high of values as you can. You can call up the list of parameters with 'ray start --help'. Additionally, the terminal that this code is executed on must also be a non-administrator terminal for this to work properly.

If you have CuDNN installed alongside TensorFlow, it will cause errors with this code. The only way to avoid this issue (that I have found so far) is to temporarily move or change the name of the folder containing your CuDNN installation so that python can not find it when loading TensorFlow. An experimental feature is in development to use this code with CuDNN by passing '--enable_gpu True' on execution, but it is buggy and I do not recommend using it. I strongly recommend that users who have CuDNN installed simply change the name of the containing folder temporarily while using this code, that is the easiest and most stable solution.

To install Carla, simply download the release suitable for your operating system, and unzip the folder to your desired location. The executable file is in this unzipped folder, no additional installation required. Launching the application will start a Carla server on your machine. Note that Carla is an absolute resource hog, and will perform at whatever level it can squeeze out of your machine. This may make it necessary to limit the amount of resources it has access to in order to keep your machine from melting during long periods of training. For PC users, adjusting the 'Max Processor State' in your advanced power settings does the trick. Use a CPU temperature guage to make sure that you are not exceeding 80 degrees Celcius. The BEST way to do this would be to install Carla and everything else on a Virtual Machine with the proper amount of resources allocated to it. This way you can do other work on the host machine while training goes on without being slowed down. Make sure you protect your virtual machine from threats if you wish to exchange files between it and your host machine, or enter credentials to your accounts from it.

All of the code being run to perform this parallel training using ray can be found in the '[ARS_Carla](ARS_Carla/)' folder of this repository. Some noteable changes that I have made to the code include:
 - I have compartmentalized the observation filter functionality which takes rolling statistics of the inputs in order to normalize them, and wrapped it in a boolean parameter 'state_filter' which defaults to False, since the VGG19 outputs are all on a standard scale between 0-10. Experimenting in training using the filter may be appropriate for future work.
 - I have added learning rate decay, and delta standard deviation (called noise in Skow's code) decay, which can be adjusted by calling --lr_decay and --std_decay parameters. This allows for more exploration early on, and favoring smaller steps over time after some initial progress has been made.
 - I have added functionality which supports initializing the ARS process with a saved policy file, so that training can be resumed at a later time. The user can pass the saved policy file in .csv (weights only) or .npz (weights only or weights plus state filter stats) format into the --policy_file parameter. The code will automatically update the saved policy file 'lin_policy_plus.npz' in the assigned --log_dir location every 10 steps by default, but this can be adjusted by setting the new --log_every parameter on execution. The number of rollouts used in each evaluation step can be set with the new --eval_rollouts.
 - I have added a parameter called --show_cam which takes in an int telling the program how many cameras you would like to display from the group of workers. It defaults to 1, but for long training periods I recommend setting it to 0 in order to save CPU overhead.

If you have not already, make sure that your Carla simulator is running, and that you have started your ray server on a separate terminal. Then, run the commands below in a terminal (without the % or ! preceding them). I have assigned values to most of the useful parameters for demonstration. Feel free to modify them. There are more parameters which can be set, which can be seen by running 'code/ars.py --help'.

*NOTE* - These lines of code must be run from a terminal to work properly. I have them written here in this notebook for the sake of demonstration. Keep in mind that the % or ! character preceding these lines is not necessary in the terminal. For clarity, see the README.md file in this repository.

In [1]:
# Change the working directory to the ARS_Carla folder in this repository
%cd ARS_Carla

C:\Users\indez\Documents\Flatiron\mod6project\ARS_Carla


In [2]:
# The authors of the 2018 paper recommend setting MKL_NUM_THREADS=1, but it is unclear whether
# this is to be done in the terminal where the ray cluster is started, or the terminal you run
# the program from. Thus, I recommend that you run 'set MKL_NUM_THREADS=1' in the terminal that
# you started the ray cluster from, and run it here as well, just to be sure
!set MKL_NUM_THREADS=1

In [8]:
# We can see a list of the optional parameters like so
!python code/ars.py --help

usage: ars.py [-h] [--env_name ENV_NAME] [--n_iter N_ITER]
              [--num_deltas NUM_DELTAS] [--deltas_used DELTAS_USED]
              [--learning_rate LEARNING_RATE] [--lr_decay LR_DECAY]
              [--delta_std DELTA_STD] [--std_decay STD_DECAY]
              [--n_workers N_WORKERS] [--rollout_length ROLLOUT_LENGTH]
              [--show_cam SHOW_CAM] [--policy_file POLICY_FILE]
              [--seconds_per_episode SECONDS_PER_EPISODE]
              [--state_filter STATE_FILTER] [--log_every LOG_EVERY]
              [--eval_rollouts EVAL_ROLLOUTS] [--shift SHIFT] [--seed SEED]
              [--policy_type POLICY_TYPE] [--dir_path DIR_PATH]
              [--filter FILTER]

optional arguments:
  -h, --help            show this help message and exit
  --env_name ENV_NAME
  --n_iter N_ITER, -n N_ITER
  --num_deltas NUM_DELTAS, -nd NUM_DELTAS
  --deltas_used DELTAS_USED, -du DELTAS_USED
  --learning_rate LEARNING_RATE, -lr LEARNING_RATE
  --lr_decay LR_DECAY, -lrd LR_DECAY
  --de

2020-12-21 20:52:38.049805: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cudart64_101.dll
2020-12-21 20:52:46,816	INFO worker.py:651 -- Connecting to existing Ray cluster at address: 169.254.86.131:6379


In [ ]:
# It is advisable to leave show_cam at 0 if you intend on running a large training session in
# order to save CPU overhead for the FPS of workers. If you want to get some visual satisfaction
# run this code with a small n_iter and set show_cam to the same value as n_workers.
# Take a moment to scroll to the right and see how parameters are set.
# NOTE: Executing this line from a Jupyter notebook will not work properly, please follow
# the directions in this notebook from a terminal window, and leave out the % and ! symbols
# preceding the command lines when you enter them.
!python code/ars.py --n_iter 1000 --num_deltas 32 --deltas_used 16 --learning_rate 0.02 --lr_decay 0.001 --delta_std 0.03 --std_decay 0.001 --n_workers 4 --show_cam 0 --seconds_per_episode 15 --log_every 10 --eval_rollouts 100 --seed 42 --dir_path .\data\old_logs\2020-12-21_test

In [ ]:
# To train a pre-existing policy, pass the location of the policy file into the 'policy_file'
# parameter:
!python code/ars.py --policy_file .\data\old_logs\2020-12-16_1250steps_part4\lin_policy_plus.npz

## Continue to next notebook
Now that we have seen how to execute the parallelized ARS training, let's move on to the next notebook, where we review, analyze, and draw conclusions from the results of the first attempt at training the agent over 1250 training steps. Click [here](notebook3_results_analysis.ipynb) to continue to the next notebook.